## Install standard libraries

In [288]:
!pip3 install pyfastx
!pip3 install biopython
!pip install scikit-plot
import numpy as np
import pickle
import torch
import torch.nn as nn
import pandas as pd
import random
from tqdm import tqdm # progress bar
from Bio.Seq import Seq
from Bio.Alphabet import generic_rna
import matplotlib.pyplot as plt

In [0]:
random.seed(42)

## Import custom scripts 

In [12]:
IMPORT = True
#IMPORT = False
if IMPORT == True:
  from google.colab import files
  uploaded = files.upload()

Saving build_dataset.py to build_dataset (1).py
Saving find_largest_gene.py to find_largest_gene (1).py
Saving remove_small_genes.py to remove_small_genes.py


In [0]:
from preproc_pipeline import window_pipeline
from warnings import simplefilter
#from cbow_model import *

In [291]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [292]:
!ls "/content/drive/My Drive"

 ari_lstm_v2.pt			 ma_rnn.pt
 cbow_kmer3_len100_epoch5.pt	'Model Outputs'
'Colab Notebooks'		 movies
 Documents			 movie_saturday2408_17.30_freepopcorn.PNG
'Documents (old stuff)'		 movie_saturday2408_17.30.PNG
 drawing_project_deep_learning	 Pictures
 Google				 sicily.gsheet
 itw.p				 sicily.xlsx
 jarubad.PNG			 wti.p
 ma_model.pt


In [293]:
%cd "/content/drive/My Drive/Model Outputs"

/content/drive/My Drive/Model Outputs


In [294]:
root = "/content/drive/My Drive/Model Outputs"
root

'/content/drive/My Drive/Model Outputs'

# Dataset
In this first phase we use the genome of E. coli.
We want to import more genome from wider range of bacteria in a second step.

## Reading fasta and tsv files.

we create a dataframe and look into it

In [295]:
%cd "/content/drive/My Drive/Model Outputs/genomes"

/content/drive/My Drive/Model Outputs/genomes


In [0]:
simplefilter("ignore")
genomes = ("GCF_000008525.1_ASM852v1_genomic.fna",
"GCF_000008665.1_ASM866v1_genomic.fna",
"GCF_000008865.2_ASM886v2_genomic.fna",
"GCF_000009045.1_ASM904v1_genomic.fna",
"GCF_000027305.1_ASM2730v1_genomic.fna",
"GCF_000027325.1_ASM2732v1_genomic.fna",
"GCF_000027345.1_ASM2734v1_genomic.fna",
"GCF_000091665.1_ASM9166v1_genomic.fna",
"GCF_000214725.1_ASM21472v1_genomic.fna",
"GCF_003015225.1_ASM301522v1_genomic.fna"
)
feature_tables = ("GCA_000008525.1_ASM852v1_feature_table.tsv",
"GCA_000008665.1_ASM866v1_feature_table.tsv",
"GCA_000008865.2_ASM886v2_feature_table.tsv",
"GCA_000009045.1_ASM904v1_feature_table.tsv",
"GCA_000027305.1_ASM2730v1_feature_table.tsv",
"GCA_000027325.1_ASM2732v1_feature_table.tsv",
"GCA_000027345.1_ASM2734v1_feature_table.tsv",
"GCA_000091665.1_ASM9166v1_feature_table.tsv",
"GCA_000214725.1_ASM21472v1_feature_table.tsv",
"GCA_003015225.1_ASM301522v1_feature_table.tsv")

In [315]:
for i in range(len(genomes)):
  c = 0
  for name, seq in pyfastx.Fasta(genomes[i], build_index = False):
    #print (type(seq))
    #print (name)
    print ('%d_%d %s %s, lenght:%d'% (i,c,genomes[i],name,len(seq)))
    c = c+1
  print (dash)

  

0_0 GCF_000008525.1_ASM852v1_genomic.fna NC_000915.1, lenght:1667867
--------------------------------------------------------------------------------
1_0 GCF_000008665.1_ASM866v1_genomic.fna NC_000917.1, lenght:2178400
--------------------------------------------------------------------------------
2_0 GCF_000008865.2_ASM886v2_genomic.fna NC_002695.2, lenght:5498578
2_1 GCF_000008865.2_ASM886v2_genomic.fna NC_002128.1, lenght:92721
2_2 GCF_000008865.2_ASM886v2_genomic.fna NC_002127.1, lenght:3306
--------------------------------------------------------------------------------
3_0 GCF_000009045.1_ASM904v1_genomic.fna NC_000964.3, lenght:4215606
--------------------------------------------------------------------------------
4_0 GCF_000027305.1_ASM2730v1_genomic.fna NC_000907.1, lenght:1830138
--------------------------------------------------------------------------------
5_0 GCF_000027325.1_ASM2732v1_genomic.fna NC_000908.2, lenght:580076
-----------------------------------------------

In [0]:
seq = pyfastx.Fasta(genomes[2], build_index = False)
print (seq[0])
print (dash)

In [410]:
import pyfastx
import pickle
GEN = dict()
splitting_loc = list()
for i in range(len(genomes)):
  c = 0
  fna= pyfastx.Fasta(genomes[i], build_index = False)
  GEN[i] = str(fna[0])
  print ('%d_%d %s '% (i,c,genomes[i]))
  print ('%s... tot lenght:%d '%(GEN[i][0:10],len(GEN[i])))
  s = int(len(GEN[i])/2)
  splitting_loc.append(s)
  training_fasta = GEN[i][0:s]
  testing_fasta = GEN[i][-s:]
  
  ofile = open("%s_%s_TRAIN_%s" % (i,c,genomes[i]), "w")
  ofile.write(">%s_%s_TRAIN_%s\n%s\n" % (i,c,genomes[i], training_fasta))
  ofile.close()
  print ('file %s_%s_TRAIN_%s loaded' %(i,c,genomes[i]))


  ofile = open("%s_%s_TEST_%s" % (i,c,genomes[i]), "w")
  ofile.write(">%s_%s_TEST_%s\n%s\n" % (i,c,genomes[i], testing_fasta))
  ofile.close()
  print ('file %s_%s_TEST_%s loaded' %(i,c,genomes[i]))
  print ()
  c = c+1

print (splitting_loc)
with open("splitting_loc.pkl", "wb") as fp:   #Pickling
  pickle.dump(splitting_loc, fp)

0_0 GCF_000008525.1_ASM852v1_genomic.fna 
TGATTAGTGA... tot lenght:1667867 
file 0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna loaded
file 0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna loaded

1_0 GCF_000008665.1_ASM866v1_genomic.fna 
GAAAATGGTT... tot lenght:2178400 
file 1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna loaded
file 1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna loaded

2_0 GCF_000008865.2_ASM886v2_genomic.fna 
AGCTTTTCAT... tot lenght:5498578 
file 2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna loaded
file 2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna loaded

3_0 GCF_000009045.1_ASM904v1_genomic.fna 
ATCTTTTTCG... tot lenght:4215606 
file 3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna loaded
file 3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna loaded

4_0 GCF_000027305.1_ASM2730v1_genomic.fna 
TATGGCAATT... tot lenght:1830138 
file 4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna loaded
file 4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna loaded

5_0 GCF_000027325.1_ASM2732

In [411]:
!ls

0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna
0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna.fxi
0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna
0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna.fxi
1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna
1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna.fxi
1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna
1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna.fxi
2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna
2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna.fxi
2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna
2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna.fxi
3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna
3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna.fxi
3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna
3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna.fxi
4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna
4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna.fxi
4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna
4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna.fxi
5_0_TEST_GCF_000027325.1_A

In [350]:
## test

for name, seq_test in pyfastx.Fasta("0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna", build_index = False):
  print (name)
  print (len(seq_test), seq_test[0:10])
#print (seq_test)

0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna
833933 TGATTAGTGA


In [351]:
with open("splitting_loc.pkl", "rb") as fp:   # Unpickling
  b = pickle.load(fp)

b

[833933,
 1089200,
 2749289,
 2107803,
 915069,
 290038,
 408197,
 832485,
 1273270,
 1787155]

attempt to edit tsv feature table

In [371]:
import csv

with open("GCA_000008525.1_ASM852v1_feature_table.tsv") as tsvfile:
  tsvreader = csv.reader(tsvfile, delimiter='\t')
  c=0
  for row in tsvreader:
        #print(row)
        c = c+1
  print (c) 


3242


In [0]:
!ls

In [0]:
simplefilter("ignore")
train_genomes = ("0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna",
"1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna",
"2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna",
"3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna",
"4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna",
"5_0_TRAIN_GCF_000027325.1_ASM2732v1_genomic.fna",
"6_0_TRAIN_GCF_000027345.1_ASM2734v1_genomic.fna",
"7_0_TRAIN_GCF_000091665.1_ASM9166v1_genomic.fna",
"8_0_TRAIN_GCF_000214725.1_ASM21472v1_genomic.fna",
"9_0_TRAIN_GCF_003015225.1_ASM301522v1_genomic.fna")

train_feature_tables = ("GCA_000008525.1_ASM852v1_feature_table.tsv",
"GCA_000008665.1_ASM866v1_feature_table.tsv",
"GCA_000008865.2_ASM886v2_feature_table.tsv",
"GCA_000009045.1_ASM904v1_feature_table.tsv",
"GCA_000027305.1_ASM2730v1_feature_table.tsv",
"GCA_000027325.1_ASM2732v1_feature_table.tsv",
"GCA_000027345.1_ASM2734v1_feature_table.tsv",
"GCA_000091665.1_ASM9166v1_feature_table.tsv",
"GCA_000214725.1_ASM21472v1_feature_table.tsv",
"GCA_003015225.1_ASM301522v1_feature_table.tsv")

In [0]:
simplefilter("ignore")
test_genomes = ("0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna",
"1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna",
"2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna",
"3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna",
"4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna",
"5_0_TEST_GCF_000027325.1_ASM2732v1_genomic.fna",
"6_0_TEST_GCF_000027345.1_ASM2734v1_genomic.fna",
"7_0_TEST_GCF_000091665.1_ASM9166v1_genomic.fna",
"8_0_TEST_GCF_000214725.1_ASM21472v1_genomic.fna",
"9_0_TEST_GCF_003015225.1_ASM301522v1_genomic.fna")

test_feature_tables = ("GCA_000008525.1_ASM852v1_feature_table.tsv",
"GCA_000008665.1_ASM866v1_feature_table.tsv",
"GCA_000008865.2_ASM886v2_feature_table.tsv",
"GCA_000009045.1_ASM904v1_feature_table.tsv",
"GCA_000027305.1_ASM2730v1_feature_table.tsv",
"GCA_000027325.1_ASM2732v1_feature_table.tsv",
"GCA_000027345.1_ASM2734v1_feature_table.tsv",
"GCA_000091665.1_ASM9166v1_feature_table.tsv",
"GCA_000214725.1_ASM21472v1_feature_table.tsv",
"GCA_003015225.1_ASM301522v1_feature_table.tsv")

In [406]:

if not len(train_genomes) == len(train_feature_tables):
  print ("tot genomes: %d; tot feature tables:%d, error: they are expected to be the same number" %(len(train_genomes),len(train_feature_tables))) 
else: 
  print ("tot genomes: %d; tot feature tables:%d" %(len(train_genomes),len(train_feature_tables))) 
  df_train = dict()
  for i in range(len(feature_tables)):
    #print (len(train_genomes[i]), len(train_feature_tables[i]))
    #print ((window_pipeline(genomes[i], feature_tables[i])))
    df_train[i] = window_pipeline(train_genomes[i], train_feature_tables[i])#<---------------------windows takes 50+50+50 from features_tables and tries to pair it with the train_genome... being the train_genome halved, many lines are empty!
    #print (type(df_train[i]))


tot genomes: 10; tot feature tables:10


In [0]:
dash = '-'*80

from each genome as minimum we have:
- 1000 genes 
- 200 intergenes
- 1000 partial genes
 

 we decide to under rapresent cat 5)

5) GCF_000027325.1_ASM2732v1_genomic.fna 
- has lenght: 3116
- Number of genes: 1431
- **Number of intergenes: 264**
- Number of partial genes: 1421

for training for this genome we take 100,100(,100) genes,intergens (and partial)genes.

Looking at the second shortest:
6) GCF_000027345.1_ASM2734v1_genomic.fna 
- has lenght: 4792
- Number of genes: 2049
- **Number of intergenes: 714**
- Number of partial genes: 2029

for training for all the other genomes we take 700 genes, 700 intergenes and eventually 700 partial genes from each genome.





In [381]:
## print one example
n =4
print (n,GEN[n])
#print (df[n]) # all dataframe
print (type(df[n]) )
print (df[n].label)#access one colum
print (list(set(df[n].label)))# unique entries for one column
#print (df[n][df[n].label== "gene"]) # how to access only genes
print (len(df[n][df[n].label== "gene"]))

4 ...
<class 'pandas.core.frame.DataFrame'>
0          gene
1          gene
2          gene
3          gene
4          gene
         ...   
5013    partial
5014    partial
5015    partial
5016    partial
5017    partial
Name: label, Length: 12213, dtype: object
['intergenic', 'partial', 'gene']
5085


In [77]:
print(
    f"columns -> {list(df[n].columns)}\n"
    f"labels in dataframe -> {list(df[n].label.unique())}"
)

columns -> ['sequence', 'label']
labels in dataframe -> ['gene', 'intergenic', 'partial']


## Deleting lines with lenght = 0

In [404]:
print(df[n][df[n].sequence.apply(lambda x: len(x)==0)].count())
df[n] = df[n][~df[n].sequence.apply(lambda x: len(x)==0)]

sequence    0
label       0
dtype: int64


We noticed there is always a line with lenght = 0, that we delete from the dataframe.

## One-hot labels encoding

In [383]:
labels = list(df[n].label.unique())
lab0 = np.zeros(len(labels))
lab2vec = {} # create empty dictionaries
vec2lab = {}
for i, label in enumerate(list(df[n].label.unique())):
    labv = lab0.copy()
    labv[i] = 1
    lab2vec[label] = labv
    vec2lab[tuple(labv)] = label

print(f"lab2vec -> {lab2vec}\nvec2lab -> {vec2lab}")

lab2vec -> {'gene': array([1., 0., 0.]), 'intergenic': array([0., 1., 0.]), 'partial': array([0., 0., 1.])}
vec2lab -> {(1.0, 0.0, 0.0): 'gene', (0.0, 1.0, 0.0): 'intergenic', (0.0, 0.0, 1.0): 'partial'}


## Create train dataframe and test dataframe

## Training dataset

In [402]:

appended_data = []
for i in range(len(df_train)):
  if not i == 5:
    data = df_train[i][df_train[i].label=="gene"].sample(n=350)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="intergenic"].sample(n=350)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="partial"].sample(n=350)
    appended_data.append(data)
  else: 
    data = df_train[i][df_train[i].label=="gene"].sample(n=100)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="intergenic"].sample(n=100)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="partial"].sample(n=100)
    appended_data.append(data)

# see pd.concat documentation for more info
training_data = pd.concat(appended_data).reset_index(drop=False)

#shuffle
training_data = training_data.sample(frac=1)
print (training_data)


      index                                           sequence       label
7038    269  GGGGAGACACTCTCTTTATAAAAATATACCTTATTTATTCTAAAAT...  intergenic
402     260  TGCGAATTGTCTAAAAAGCGTATTTTAGAGAGTTTGTCGTTAGTGT...  intergenic
6092     45  ATACCTACTTTGAGCCAACACGGTTCGAGTACGGATCCGAGTTTTG...  intergenic
2909   9659                                                        partial
7385    300  CTATAATTTCCTTCTCTTCAAAGCCGTTTATCATTTGAGGGACTAA...     partial
...     ...                                                ...         ...
7283    868  TTGATTTTAGATAGTATATATAACTTTCTAAATTTTTTAAATAGAA...  intergenic
1974   6815                                                        partial
2768   3488                                                     intergenic
586      16  AGATAATAAAGACTTATAAAGTTAATTAAAATTAAACAGAAGGGTT...  intergenic
9390    231  ACCGCAAGGGGTCAAACCACCATGGCCGCAGATATCTTACACCGGA...  intergenic

[9750 rows x 3 columns]


## Testing dataset

In [407]:
if not len(test_genomes) == len(test_feature_tables):
  print ("tot genomes: %d; tot feature tables:%d, error: they are expected to be the same number" %(len(train_genomes),len(train_feature_tables))) 
else: 
  print ("tot genomes: %d; tot feature tables:%d" %(len(train_genomes),len(train_feature_tables))) 
  df_test = dict()
  for i in range(len(feature_tables)):
    #print (len(train_genomes[i]), len(train_feature_tables[i]))
    #print ((window_pipeline(genomes[i], feature_tables[i])))
    df_test[i] = window_pipeline(test_genomes[i], test_feature_tables[i])#<---------------------windows takes 50+50+50 from features_tables and tries to pair it with the test_genome... being the train_genome halved, many lines are empty!
    #print (type(df_train[i]))

tot genomes: 10; tot feature tables:10


In [412]:

appended_data = []
for i in range(len(df_test)):
  if not i == 5:
    data = df_test[i][df_test[i].label=="gene"]
    data = data[0:350]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="intergenic"]
    data = data[0:350]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="partial"]
    data = data[0:350]
    appended_data.append(data)
  else: 
    data = df_test[i][df_test[i].label=="gene"]
    data = data[0:100]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="intergenic"]
    data = data[0:100]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="partial"]
    data = data[0:100]
    appended_data.append(data)

# see pd.concat documentation for more info
testing_data = pd.concat(appended_data).reset_index(drop=False)

#no shuffle
#testing_data = testing_data.sample(frac=1)
print (testing_data)


      index                                           sequence    label
0         0  CAAGAGCTTATTAATGAGGGCTATTATGCGATTTTTGGGGCTGCCG...     gene
1         1  CTTTACTAGGGCGAATCCCCACTAAAGAAGAATACATGAATTTAGT...     gene
2         2  TTAATTATGGCTGAGCGTTGAGAGAAAATAAAAAAGGAGAGTGGCG...     gene
3         3  AAAAATCGTTGTGAGTTGGTGTGTGGCGTTGGCTTTTTTAAGCGCG...     gene
4         4  AGGCCTTATATCTATAATTGGCGCTAGGGGATAATCCAAAAAATGA...     gene
...     ...                                                ...      ...
9745    345  GCCAATGTTTACGTGGTCTTTCGTCCGTTCAAATTTTGCGCGTGCC...  partial
9746    346  ATGGAACACTAGCGGCCACTTTGGCTATGCCCTGCTCCGTTTCCTG...  partial
9747    347  CACCGACGATTTTAGAGACAAATTCAAAGCCCGTCCCTGGCTCGGT...  partial
9748    348  AAAGATTGTAAAGCTTTGGAAAGCTTATCCATATCCTGTTTAGTTT...  partial
9749    349  GCGGTCATCGGCTTTGAGGATGATTAATCGGGAAATGCGTTCCTTC...  partial

[9750 rows x 3 columns]
